2025-10-12 23:43:17 Sunday
### 你能讲讲 GRPO（Group Relative Policy Optimization）的核心思想吗?
GRPO 用 **组内相对奖励差异** 替代 PPO 的 critic。  
对每个输入 query，从旧策略生成多条回答 \(y_i\)，计算奖励 \(r_i\)，并在组内标准化：

$$
\hat A_i = \frac{r_i - \text{mean}(r)}{\text{std}(r)}
$$

得到相对优势。优化目标是最大化高 reward 回答的生成概率：

$$
J_{\text{GRPO}}(\theta)
= \mathbb E_{q,\{o_i\}}\Bigg[
\frac{1}{G}\sum_{i,t}
\min\big(r_{i,t}\hat A_{i,t},\,\text{clip}(r_{i,t},1-\varepsilon,1+\varepsilon)\hat A_{i,t}\big)
-\beta\,D_{\mathrm{KL}}(\pi_\theta\|\pi_{\text{ref}})
\Bigg]
$$

其中：

$$
r_{i,t} = \frac{\pi_\theta(o_{i,t}|q,o_{i,<t})}{\pi_{\text{old}}(o_{i,t}|q,o_{i,<t})}
$$

优势函数乘的项是 **概率比值**，而不是策略熵。  
策略熵可选作正则项以鼓励探索多样性：

$$
H(\pi) = -\sum_x \pi(x) \log \pi(x)
$$


#### 追问：KL 项的作用？

1️⃣ **限制策略分布漂移** —— 防止更新过猛、训练不稳定；  
2️⃣ **保持语言与参考模型一致** —— 防止语义退化、语气异常；  
3️⃣ **与熵项协同** —— 共同抑制模式塌陷（即模型输出单一模板化）。


#### 追问：与 PPO 的区别？

- 无需 critic（value 网络）；  
- 优势来自 **组内归一化**，而非时序估计 (GAE)；  
- KL 惩罚作为 **独立正则项**，不混入 reward；  
- 对组大小敏感：组太小方差大，组太大梯度稀释。

